   # Analiza liniowego modelu wyborcy na pierścieniu

In [2]:
using DelimitedFiles
using Plots
using LaTeXStrings
using PGFPlotsX

# 1.Funkcje pomocnicze:

## 1.1 Funkcja do generowania opini
Zależna od długości wektora N oraz $\Delta$x

In [3]:
function generator_opini(N,dx)
    macierz=ones(N)*(-1) #generujemy macierz z negatywnymi opiniami
    
    while count(i->(i==1),macierz)<dx*N #przekształcamy macierz tak aby ilosc pozytywnych opini byla rowna dx*N
        k=rand(1:N)
        macierz[k]=1
    end
    return macierz
end

generator_opini (generic function with 1 method)

## 1.2. Funkcja do wyliczania prawdopodobieństwa pozytywnego konsensusu (dla pojedyńczej wartości x), oraz czasu uporządkowania
Zależna od parametru N (długości wektora), x(koncentracja pozytywnej opini), L (ilości kroków Monte Carlo)

In [4]:
function analiza(N,x,L)
    konsensus=0
    T=0
    for i=1:L
        czasMC=0
        agenci=generator_opini(N,x) #agenci z opiniami
        start=1
        while start==1 #losujemy tak dlugo az nasz stan uporzadkuje sie w jedna strone
            for i=1:N
                sluchajacy=rand(1:N) #losujemy słuchającego z agentów

                if sluchajacy == 1 #jeśli wylosowany sluchajacy bedzie pierwszy
                    sasiad=rand([N,sluchajacy+1]) #sasiadem bedzie 2 element w liscie albo ostatni
                elseif sluchajacy == N #jesli wylosowany sluchajacy bedzie ostatni
                    sasiad=rand([1,sluchajacy-1])
                else
                    sasiad=rand([sluchajacy-1,sluchajacy+1])
                end
                agenci[sluchajacy]=agenci[sasiad] #opinia sluchajacego staje sie opinia sasiada
            end
            
            if agenci==ones(N) || agenci==ones(N)*(-1)
                start=0
                T+=czasMC
            end
            czasMC+=1
        end
        
        if agenci==ones(N) #sprawdzamy czy nasz stan się uporządkował pozytywnie
           konsensus+=1
        end
    end
    prawdopodobienstwo=konsensus/L
    return x,prawdopodobienstwo,T/L 
end

analiza (generic function with 1 method)

# 2.Funkcja główna

## 2.Funkcja do wyliczania prawdopodobieństwa pozytywnego konsensusu (na przedziale [0,1] z zadanym krokiem $\Delta$x), oraz czasu uporządkowania (jako wynik zwraca plik tekstowy z 3 kolumnami)
Zależna od parametru N (długości wektora), $\Delta$x, L (ilości kroków Monte Carlo)

In [5]:
function analiza_glowna(N,dx,L)
    xs=0:dx:1
    open("N$(N)_dx$(dx)_L$(L).txt", "w")
    for x in xs
        wartosci=analiza(N,x,L)
        open("N$(N)_dx$(dx)_L$(L).txt", "a") do io
                   writedlm(io, [wartosci])
        end
    end
end

analiza_glowna (generic function with 1 method)

## Symulacje dla  L=500, N=100,200,300 oraz $\Delta$x=0.05

In [5]:
dx=0.05
L=500

N1=100
N2=200
N3=300

analiza_glowna(N1,dx,L)
analiza_glowna(N2,dx,L)
analiza_glowna(N3,dx,L)

## Odczytujemy nasze wyliczone wartości z pliku oraz tworzymy wykresy

In [6]:
X1=readdlm("N100_dx0.05_L500.txt");
X2=readdlm("N200_dx0.05_L500.txt");
X3=readdlm("N300_dx0.05_L500.txt");

In [12]:
pgfplotsx()
plot(X1[:,1],X1[:,2],xlabel="\$\\frac{N_+(0)}{N}\$",markershape = :auto,title="Wykres prawdopodobieństwa konsensusu od koncentracji",
ylabel= "\$P_+\$",label=["N=100"])
plot!(X2[:,1],X2[:,2],markershape = :auto,label=["N=200"])
plot!(X3[:,1],X3[:,2],markershape = :auto,label=["N=300"])
savefig("wykres1.png")

In [13]:
pgfplotsx()
plot(X1[:,1],X1[:,3],ylabel="\$\\tau\$",markershape = :auto,title="Wykres czasu dochodzenia do konsensusu od koncentracji",
xlabel= "\$\\frac{N_+(0)}{N}\$",label=["N=100"])
plot!(X2[:,1],X2[:,3],markershape = :auto,label=["N=200"])
plot!(X3[:,1],X3[:,3],markershape = :auto,label=["N=300"])
savefig("wykres2.png")

# Symulacje dla L= $10^3$, $\Delta$x=0.02 oraz N=$100 $,$250$,$500$

In [14]:
dx=0.02
L=10^3

N1=100
N2=250
N3=500

analiza_glowna(N1,dx,L)
analiza_glowna(N2,dx,L)
analiza_glowna(N3,dx,L)

In [15]:
X1=readdlm("N100_dx0.02_L1000.txt");
X2=readdlm("N250_dx0.02_L1000.txt");
X3=readdlm("N500_dx0.02_L1000.txt");

In [17]:
pgfplotsx()
plot(X1[:,1],X1[:,2],xlabel="\$\\frac{N_+(0)}{N}\$",markershape = :auto,title="Wykres prawdopodobieństwa konsensusu od koncentracji",
ylabel= "\$P_+\$",label=["N=100"])
plot!(X2[:,1],X2[:,2],markershape = :auto,label=["N=250"])
plot!(X3[:,1],X3[:,2],markershape = :auto,label=["N=500"])
savefig("wykres1.png")

In [19]:
pgfplotsx()
plot(X1[:,1],X1[:,3],ylabel="\$\\tau\$",markershape = :auto,title="Wykres czasu dochodzenia do konsensusu od koncentracji",
xlabel= "\$\\frac{N_+(0)}{N}\$",label=["N=100"])
plot!(X2[:,1],X2[:,3],markershape = :auto,label=["N=250"])
plot!(X3[:,1],X3[:,3],markershape = :auto,label=["N=500"])
savefig("wykres2.png")